In [1]:
#### Analysis focusing on the non-EU countries
#### and non-big players – that is, no Russia, Ukraine, China, US

In [2]:
import pandas as pd
import pycountry

In [3]:
# Data
df = pd.read_feather("../../output/for-analysis.feather")

In [4]:
# EU members plus USA, China, Russia, Ukraine
big_players = [ "RUS", "UKR", "USA", "CHN" ]

eu_members = [
    "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", 
    "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", 
    "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE",    
]

In [5]:
# All others (non-eu + non-big players)
all_others = [ item for item in df.columns.tolist()[17:] if item not in big_players and item not in eu_members ]

# All non-eu members
non_eu = [ item for item in df.columns.tolist()[17:] if item not in eu_members ]

In [8]:
df[big_players + eu_members + non_eu].sum().drop_duplicates().sort_values(ascending=False).head(50)

UKR    5510
RUS    4366
CHN    2574
POL    2420
FRA    1896
HUN    1831
DEU    1662
ESP    1546
USA    1533
ITA    1515
TUR    1510
GRC    1136
ROU    1045
IRL     948
IRN     850
BGR     784
BLR     754
SYR     695
GBR     669
SWE     622
PRT     588
MDA     563
AFG     550
NLD     489
MLT     487
HRV     458
BEL     454
IND     450
LBY     430
THA     407
GEO     401
SVK     392
AZE     377
AUT     360
VEN     354
PSE     344
CYP     324
SRB     321
LTU     307
MKD     298
IRQ     293
FIN     291
MLI     287
BRA     284
ARM     283
ALB     279
DNK     278
QAT     275
TUN     257
SVN     256
dtype: int64

In [ ]:
# Countries with the most mentions among all others
df[all_others].sum().sort_values(ascending=False).head(50)

In [ ]:
# Same, but by share of total speeches
(df[all_others].sum().sort_values(ascending=False) / df.shape[0] * 100)

In [ ]:
# Eu: how much of total?
df[eu_members].sum().sum() / df.shape[0] * 100

In [ ]:
# Non eu: how much of total?
df[non_eu].sum().sum() / df.shape[0] * 100

In [ ]:
# Big-players: how much of total?
df[big_players].sum().sum() / df.shape[0] * 100

In [ ]:
# Russia and Ukraine, how much of total?
df[['RUS', 'UKR']].sum().sum() / df.shape[0] * 100

In [ ]:
# Usa and China, how much of total?
df[['USA', 'CHN']].sum().sum() / df.shape[0] * 100

In [ ]:
# China, how much of total?
df[['CHN']].sum().sum() / df.shape[0] * 100

In [ ]:
# I knew that Thailand was full of false positives before. Is this still the case?
for index, row in df[df.THA].iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)

In [ ]:
# It's a common language in Baltic languages.
# If we exclude them from the count, it's not in the top 10
# We will not mention it on the story.
df[df.THA].language.value_counts()

In [ ]:
# India has a similar issue with the word 'indien', a common dutch term.
# However, it still stays high on top if we ignore the language.
df[df.IND].language.value_counts()

In [ ]:
# Manual inspection of samples of all other highlights gave us no clear indication of any other event like this.

In [ ]:
# Total share of speeches that touch on each topic
df[df.columns[8:17]].mean()

In [ ]:
### Manually inspecting a share of each country-topic pair of interest

In [ ]:
#### UK
for index, row in df[(df.GBR) & (df['International Relations and Diplomacy'])].head(10).sample(frac=1).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)

In [ ]:
# India and Economy
for index, row in df[(df.IND) & (df['Economy, Finance and Infrastructure'])].sample(frac=1).head(10).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)
    print()

In [ ]:
df[(df.IND) & (df['Environment and Climate'])]

In [ ]:
# India and Economy
for index, row in df[(df.IND) & (df['Environment and Climate'])].sample(frac=1).head(10).iterrows():
    print(row.language)
    print(row.valid_tags)
    print(row.speech)
    print()

#### Exporting data for charts

In [ ]:
# Export for bar chart:
# Non-EU countries
def make_bar_chart(df, non_eu, eu_members):
    
    def get_country(iso3):
        if iso3 == 'IRN':
            country_name = 'Iran'

        elif iso3 == 'RUS':
            country_name = 'Russia'

        elif iso3 == 'SYR':
            country_name = 'Syria'

        elif iso3 == 'COD':
            country_name = 'DR Congo'

        elif iso3 == 'TUR':
            country_name = 'Turkey'
            
        elif iso3 == 'MDA':
            country_name = 'Moldova'
        
        else:
            country_name = pycountry.countries.get(alpha_3=iso3).name
            
        return country_name
    
    
    def get_iso2(iso3):
        return pycountry.countries.get(alpha_3=iso3).alpha_2.lower()

    data = df[non_eu].sum().sort_values(ascending=False).head(14).reset_index().rename(columns={'index':'iso3',
                                                                                                 0: 'count'})
    
    data['percentage'] = data['count'] / df.shape[0] * 100
    data['country'] = data.iso3.apply(get_country)
    data['iso2'] = data.iso3.apply(get_iso2)
    data['display_name'] = ":" + data.iso2 + ":" + " " + data.country
    
    return data

In [ ]:
make_bar_chart(df, non_eu, eu_members).to_csv("../../viz-files/non-eu-ranking.csv", index=False)

#### Countries + big topics

In [ ]:
# of_interest = df[all_others].sum().sort_values(ascending=False).head(10).index
# topics = df.columns[9:17].tolist()

# rows = []
# for country in of_interest:
    
        
#     subset = df[df[country]]
#     print(country)
    
#     if country == 'IRN':
#         #print('yoyo')
#         country_name = 'Iran'
        
#     elif country == 'RUS':
#         #print('yoyo')
#         country_name = 'Russia'
        
#     elif country == 'SYR':
#         #print('yoyo')
#         country_name = 'Syria'
        
#     elif country == 'COD':
#         #print('yoyo')
#         country_name = 'DR Congo'
        
#     elif country == 'TUR':
#         #print('yoyo')
#         country_name = 'Turkey'
        
#     elif country == 'MDA':
#         country_name = 'Moldova'
        
        
#     else:
#         country_name = pycountry.countries.get(alpha_3=country).name
        
#     print(country_name)
    
#     for topic in topics:
#         row = {}
        
#         # % about the issue
#         share = subset[topic].sum() / subset.shape[0] * 100
#         row['iso3'] = country
#         row['iso2'] = pycountry.countries.get(alpha_3=country).alpha_2
#         row['country'] = f":{row['iso2'].lower()}: {country_name}"
#         row['value_country'] = share
# #         row['% of all speeches that menion the topic'] = df[topic].sum() / df.shape[0] * 100
#         #row['value_total'] = df[topic].sum() / df.shape[0] * 100
#         row['topic'] = topic.lower()
#         row['label'] = f"Of all speeches that mention ___, how many are about {topic.lower()}?"
#         row['count_speeches'] = subset.shape[0]
        
#         rows.append(row)
        
# # Adds the overall value for all speeches
# for topic in topics:
#     row = {}
#     row['iso3'] = 'ALL'
#     row['country'] = 'All speeches'
#     row['topic'] = topic.lower()
#     row['label'] = f"Of all speeches that mention ___, how many are about {topic.lower()}?"
#     row['value_country'] = df[topic].sum() / df.shape[0] * 100
#     row['count_speeches'] = df.shape[0]
#     rows.append(row)

In [ ]:
# topics_countries = pd.DataFrame(rows)

In [ ]:
# for topic ain topics:
#     subset = topics_countries[topics_countries['topic']==topic.lower()]
#     subset.to_csv(f"../../viz-files/{ topic }-of-interest-bars.csv", index=False)

#### By continent, with EU and non-EU split

In [ ]:
# Simple resampler for plotting data over time, using a 2 month window
def resampler(df, window='2M', agg='mean'):

    # Ensure the 'date' column is in datetime format
    df['date'] = pd.to_datetime(df['date'])

    # Set the 'date' column as the index
    df = df.set_index('date')

    # Resample the dataframe to three-month intervals and calculate the share of True values (the mean works
    # because a True is counted as 1 and a False as 0). Summing all the 1s and dividing by total observations
    # equals to a percentage of total.
    if agg == 'mean':
        df = df.resample(window).agg(agg) * 100
    else:
        df = df.resample(window).agg(agg)
        
    # Custom date format function
    def custom_date_format(index):
        
        formatted_dates = []
        
        for date in index:
            
            start_date = date - pd.DateOffset(months=1)
            end_date = date
            formatted_dates.append(f"{start_date.strftime('%B %Y')} to {end_date.strftime('%B %Y')}")
            
        return formatted_dates
        
    # Adds a human readable tag for the datawrapper legend
    df['span'] = custom_date_format(df.index)
    
#     df.iloc[0, 'span'] = 'July 2019'
#     df.iloc[-1, 'span'] = 'April 2024'
    
    return df

In [ ]:
# Continental overview – ISO3 to continent
correspondence = pd.read_csv("../../input/country-and-continent-codes-list.csv", sep=';')

In [ ]:
# Keep only the iso codes that exist in the columns
correspondence = correspondence[correspondence['ISO-alpha3 code'].isin(df.columns)]

In [ ]:
# A dict with the matches
continent_iso_map = correspondence.groupby('continent')['ISO-alpha3 code'].apply(list).to_dict()

In [ ]:
continent_iso_map['European Union'] = eu_members

In [ ]:
continent_iso_map['Europe, non-EU'] = [ country for country in continent_iso_map['Europe'] if country not in eu_members]

In [ ]:
# Adds a column to check whether any country in the continent was mentioned
for continent, countries in continent_iso_map.items():
    df[continent] = df[['date'] + countries].sum(axis=1) > 0

In [ ]:
resampler(df[['date', 'Africa', 'Americas', 'Asia', 'European Union', 'Europe, non-EU', 'Oceania']])

In [ ]:
resampler(df[['date', 'Africa', 'Americas', 'Asia', 'European Union', 'Europe, non-EU', 'Oceania']])\
    .reset_index()\
    .to_csv("../../viz-files/continents_2m_mean_eu_split.csv", index=False)

#### Topics over time

In [ ]:
topics = df.columns[8:17].tolist()
resampler(df[ ['date'] + topics]).to_csv("../../viz-files/all-topics-multiple-bars.csv", index=False)

#### Selected countries + sortable table

In [ ]:
groups = ['Parliament issues and procedures',
 'Democracy and Governance',
 'War and Security',
 'International Relations and Diplomacy',
 'Economy, Finance and Infrastructure',
 'Environment and Climate',
 'Health',
 'Technology and Innovation',
 'Human Rights, Freedoms and Social Issues']

In [ ]:
top_10 = df[all_others].sum().sort_values(ascending=False).head(10).index
rows = []
for country in top_10:
    
    row = {}
    
    subset = df[df[country]]
    
    if country == 'IRN':
        country_name = 'Iran'

    elif country == 'RUS':
        country_name = 'Russia'

    elif country == 'SYR':
        country_name = 'Syria'

    elif country == 'COD':
        country_name = 'DR Congo'

    elif country == 'TUR':
        country_name = 'Turkey'

    elif country == 'MDA':
        country_name = 'Moldova'
        
    else:
        country_name = pycountry.countries.get(alpha_3=country).name
    
    iso2 = pycountry.countries.get(alpha_3=country).alpha_2.lower()
    row['country_name'] = country_name
    row['display_name'] = f':{iso2}: {country_name}'
    row['category'] = 'country'
    
    for group in groups:
        
        # % about the issue
        share = subset[group].sum() / subset.shape[0] * 100
        row[group] = round(share, 2)
        
    rows.append(row)
        
        
    
# All speeches
row = {}
row['country_name'] = "All speeches"
row['display_name'] = "All speeches"
row['category'] = 'all'


for group in groups:

    # % about the issue
    share = df[group].sum() / df.shape[0] * 100
    row[group] = round(share, 2)


rows.append(row)

In [ ]:
pd.DataFrame(rows).to_csv("../../viz-files/table.csv", index=False)

In [ ]:
pd.DataFrame(rows)

#### Choropleth map with all countries

In [ ]:
#df.sum().mean()

In [ ]:
choropleth_data = (df[df.columns[17:212]].sum() / df.shape[0] * 100).reset_index()

In [ ]:
choropleth_data = choropleth_data.rename(columns={'index': 'iso3', 0: 'share'})

In [ ]:
def get_country(iso3):
    if iso3 == 'IRN':
        country_name = 'Iran'

    elif iso3 == 'RUS':
        country_name = 'Russia'

    elif iso3 == 'SYR':
        country_name = 'Syria'

    elif iso3 == 'COD':
        country_name = 'DR Congo'

    elif iso3 == 'TUR':
        country_name = 'Turkey'

    elif iso3 == 'MDA':
        country_name = 'Moldova'

    else:
        country_name = pycountry.countries.get(alpha_3=iso3).name

    return country_name

In [ ]:
choropleth_data['country_name'] = choropleth_data.iso3.apply(get_country)

In [ ]:
choropleth_data.to_csv("../../viz-files/choropleth.csv", index=False)